In [ ]:
import requests
import json

api_url = "https://api.ce-cotoha.com/api/dev"
auth_url = "https://api.ce-cotoha.com/v1/oauth/accesstokens"
client_id = "[client id]"
client_secret = "[client secret]"

## Barer tokenの取得

In [ ]:
def post_url(url, body, access_token=None):
    headers = {
        "Content-Type": "application/json",
        "charset": "UTF-8"
    }
    
    if access_token is not None:
        headers["Authorization"] = f"Bearer {access_token}"
    
    data = json.dumps(body)
    res = requests.post(url, headers=headers, data=data)
    return res

In [ ]:
def post_api(url, body, access_token=None):
    res = post_url(url, body, access_token)
    if res.status_code == 200:
        result = json.loads(res.text)
    else:
        result = None
    return result

In [ ]:
def get_access_token(url, client_id, client_secret):
    body= {
      "grantType": "client_credentials",
      "clientId": client_id,
      "clientSecret": client_secret
    }
    res = post_url(url, body)
    
    if res.status_code == 201:
        entry = json.loads(res.text)
        access_token = entry["access_token"]
    else:
        access_token = None        
    return access_token

In [ ]:
access_token = get_access_token(auth_url, client_id, client_secret)
print(access_token)

In [ ]:
def tokenize(text, api_url, access_token):
    body= {
        "sentence": text,
        "type": "default"
    }
    url = f"{api_url}/nlp/v1/parse"
    result = post_api(url, body, access_token)
    return result

In [ ]:
result = tokenize("今日は会社に行かなかった", api_url, access_token)
if result is not None:
    for chunk in result["result"]:
        for token in chunk["tokens"]:
            print(token["form"])

In [ ]:
def ner(text, api_url, access_token):
    body= {
        "sentence": text,
        "type": "default"
    }
    url = f"{api_url}/nlp/v1/ne"
    result = post_api(url, body, access_token)
    return result

In [ ]:
result = ner("今日は会社に行かなかった", api_url, access_token)
if result is not None:
    for entry in result["result"]:
        print(entry["std_form"], entry["class"], entry["extended_class"])

In [ ]:
def keyword_extraction(texts, api_url, access_token):
    body= {
        "document": texts,
        "type": "default"
    }
    url = f"{api_url}/nlp/v1/keyword".format(api_url)
    result = post_api(url, body, access_token)
    return result

In [ ]:
texts = ["会社敷地内で、被災者はミキサー車を整備中に、車体に上ろうとしたところ体のバランスを崩して転倒し被災した。"]
result = keyword_extraction(texts, api_url, access_token)
if result is not None:
    for entry in result["result"]:
        print(entry["form"],entry["score"])

In [ ]:
def sentiment_analysis(text, api_url, access_token):
    body= {
        "sentence": text
    }
    url = f"{api_url}/nlp/v1/sentiment"
    result = post_api(url, body, access_token)
    return result

In [ ]:
text = ("パラリンピック、見応えある競技も多いけど、今のところ一番はブラインドサッカーだな。"
        "盲目の剣士とか武術家はフィクションでは見るけど、リアルに音だけであんなに高度なサッカーできるとは思わなかった。"
        "超人スポーツみがあって、厨二心もくすぐられる。")
result = sentiment_analysis(text, api_url, access_token)
if result is not None:
    entry = result["result"]
    print(entry["sentiment"], entry["score"])
    for phrase in entry["emotional_phrase"]:
        print(phrase)